#### Initialize

In [ ]:
# Powershell
    $PSVersionTable

In [ ]:
$public_ip = "192.168.0.21"

In [ ]:
$keyname = "ssh"

In [ ]:
$user = "user1"

In [ ]:
cd ..\bash\

In [ ]:
$folder = Get-Location

In [ ]:
$session = New-PSSession -HostName $public_ip -UserName $user -KeyFilePath $folder\key\$keyname.pem -SSHTransport

In [ ]:
# Function to execute single ps command

    function sh ($command) {
        $scriptBlock = {
            param($command)
            Invoke-Expression $command
        }

        Invoke-Command -Session $session -ScriptBlock $scriptBlock -ArgumentList $command
    }

In [ ]:
# Function to execute single ssh command with sudo

    function sush ($command) {

        $password = Read-Host "Enter your password"

        $scriptBlock = {
            param($password, $command)
            echo $password | sudo -S bash -c $command
        }

        Invoke-Command -Session $session -ScriptBlock $scriptBlock -ArgumentList $password, $command
    }

In [ ]:
# Function to copy a file to remote host

    function shcp ($file) {
        Copy-Item $file -Destination "\home\$user" -ToSession $Session
    }

In [ ]:
# Function to copy a file from remote host

    function shget ($file) {
        Copy-Item -FromSession $session -Path $file
    }

#### General commands

In [ ]:
# Start interactive session
# Command has to be pasted from clipboard, calling pwsh with arguments does not work for some reason

    $command = "Enter-PSSession -HostName $public_ip -UserName $user -KeyFilePath '$folder\key\$keyname.pem' -SSHTransport"
    Set-Clipboard $command
    start pwsh

#start pwsh -ArgumentList "-command $command"
#start pwsh -ArgumentList "-command", $command

In [ ]:
#Execute single command
    sh "Get-ChildItem test.sh"

In [ ]:
# Run sudo command
    sush "ls -l"
    sush "pwsh -command 'Get-ChildItem test.sh'"

#### Working with files

In [ ]:
# Set file name
    $file = "test.sh"

In [ ]:
# Copy file to remote host user home folder
    shcp $file

In [ ]:
# Get file from remote host
    shget "/home/user1/$file"

In [ ]:
# Make file executable
    sh "chmod +x $file"

In [ ]:
# Run file as root
    sush "./$file"

#### Custom commands

In [ ]:
sh "ls"